In [ ]:
import numpy as np
import pandas as pd
import os
import h5py
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

In [ ]:
total_path = r'Z:\zym\IS\TH_chr2_3'
activate_path = total_path + '\\'+'activate'+'\\'+'plx_2'
filelist_a = os.listdir(activate_path)
plx_a=[]
for f in filelist_a:
    if f.endswith('.plx'):
        plx_a.append(f[:-4])

In [ ]:
ptd_all = []
wave_data = []
unit_name = []
for idx,file_name in enumerate(plx_a):
    wave = pd.read_excel(os.path.join(activate_path, file_name+'waveform.xlsx'),engine='openpyxl')
    wave = wave[wave.peak_trough_distance != 0]
    wave_data.extend(wave.values[:,0:3].tolist())
    unit_name.extend((wave.values[:,0]).tolist())
unit_name = list(dict.fromkeys(unit_name))
unit_name.sort()
for u in unit_name:
    ptd = []
    for w in wave_data:
        if w[0] == u:
            ptd.append(w[2])
    ptd_all.append(np.mean(ptd))
ptd_avg = np.array(ptd_all)
narrow_idx = np.argwhere((ptd_avg<0.4)|(ptd_avg==0.4)).flatten()
narrow_idx = narrow_idx.tolist()
broad_idx = np.argwhere(ptd_avg>0.4).flatten()
broad_idx = broad_idx.tolist()

In [ ]:
unit_narrow = []
for i in narrow_idx:
    unit_narrow.append(unit_name[i])
unit_broad = []
for i in broad_idx:
    unit_broad.append(unit_name[i])

In [ ]:
wave_a = pd.read_excel(os.path.join(activate_path, plx_a[0]+'waveform.xlsx'),engine='openpyxl')
timestamp_a = h5py.File(os.path.join(activate_path, plx_a[0]+'timestamp.mat'),'r')
#timestamp_b = h5py.File(os.path.join(baseline_path, plx_b[0]+'timestamp.mat'),'r')
kbd_a = timestamp_a['timestamp/kbd1'][:].flatten()
kbd_a

## activate-5trial-15s15s

In [ ]:
marker_a = np.arange(15,151,30)+kbd_a
unit_list = list(wave_a.unit)
unit_name = []
gap = 15
for i in unit_list:
    name = 'timestamp/'+i
    unit_name.append(name)
    fig,ax = plt.subplots(2,1,figsize=(6,4))
    spike_a = timestamp_a[name][:].flatten()
    spk_list=[]
    for m in marker_a:
        start = m - gap
        stop = m + gap
        spk = spike_a[(spike_a>start) & (spike_a<stop)]-start
        spkl = spk.tolist()
        spk_list.append(spkl)
    spk_array = np.asarray(spk_list)
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].spines['left'].set_visible(False)
    ax[0].axes.xaxis.set_visible(False)
    ax[0].eventplot(spk_array, 'horizontal',linelengths=1,linewidths=0.2,color='slategray')
    ax[0].set_ylabel('#Trial',fontsize=14)
    ax[0].set_ylim(-0.5,4.5)
    ax[0].set_yticks(np.arange(0,4.1,1))
    ax[0].set_xlim(0,2*gap)
    ax[0].set_xticks(np.linspace(0,(2*gap/0.2-1)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[0].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    ax[0].set_title(i)

    _hist = np.zeros((len(spk_array), len(np.arange(0, 2*gap, 0.2))-1))
    for row in range(len(spk_array)):
        spk_row = spk_array[row]
        distribution,_ = np.histogram(spk_row, np.arange(0, 2*gap, 0.2))
        _hist[row] = distribution
    hist = np.sum(_hist, 0)
    ax[1].bar((np.arange(0, 2*gap, 0.2)-0.1)[1:], hist, width = 0.2,color='slategray')
    title = '%s_PSTH_5trial.jpg'% i
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['bottom'].set_linewidth('1')
    ax[1].spines['left'].set_linewidth('1')
    ax[1].set_ylabel('Counts',fontsize=14)
    ax[1].set_xlim(0,2*gap)
    ax[1].set_xticks(np.linspace(0,len(hist)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[1].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    plt.savefig(os.path.join(activate_path, title),dpi=400)
    #plt.show()
    plt.close()

## activate-1trial-15s-15s

In [ ]:
gap = 15
marker_a = 15 + kbd_a
unit_list = list(wave_a.unit)
unit_name = []
for idx,i in enumerate(unit_list):
    fig,ax = plt.subplots(2,1,figsize=(6,4))
    name = 'timestamp/'+i
    unit_name.append(name)
    spike_a = timestamp_a[name][:].flatten()
    spk_list=[]
    start = marker_a - gap
    stop = marker_a + gap
    spk_array_b = spike_a[(spike_a>start) & (spike_a<start+gap)]-start
    spk_array_a = spike_a[(spike_a>start+gap) & (spike_a<stop)]-start
    spk_array = np.concatenate((spk_array_b,spk_array_a),axis=0)
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].spines['left'].set_visible(False)
    ax[0].axes.xaxis.set_visible(False)
    ax[0].eventplot(spk_array, 'horizontal',linelengths=1,linewidths=0.2,color='slategray')
    #ax[0].set_ylabel('#Trial',fontsize=14)
    #ax[0].set_ylim(-0.5,0.5)
    ax[0].set_yticklabels(['1','',''])
    ax[0].set_xlim(0,2*gap)
    ax[0].set_xticks(np.linspace(0,(2*gap/0.2-1)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[0].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    ax[0].set_title(i)

    _hist = np.zeros((len(spk_array), len(np.arange(0, 2*gap, 0.2))-1))
    for row in range(len(spk_array)):
        spk_row = spk_array[row]
        distribution,_ = np.histogram(spk_row, np.arange(0, 2*gap, 0.2))
        _hist[row] = distribution
    hist = np.sum(_hist, 0)
    ax[1].bar((np.arange(0, 2*gap, 0.2)-0.1)[1:], hist, width = 0.2,color='slategray')
    title = '%s_PSTH.jpg'% i
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['bottom'].set_linewidth('1')
    ax[1].spines['left'].set_linewidth('1')
    ax[1].set_ylabel('Counts',fontsize=14)
    ax[1].set_xlim(0,2*gap)
    ax[1].set_xticks(np.linspace(0,len(hist)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[1].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    plt.savefig(os.path.join(activate_path, title),dpi=400)
    plt.close()

## activate-5trial-2s/5s-20s

In [ ]:
marker_a = np.arange(20,111,22)+kbd_a
unit_list = list(wave_a.unit)
unit_name = []
gap = 10
for i in unit_list:
    name = 'timestamp/'+i
    unit_name.append(name)
    fig,ax = plt.subplots(2,1,figsize=(6,4))
    spike_a = timestamp_a[name][:].flatten()
    spk_list=[]
    for m in marker_a:
        start = m - gap
        stop = m + gap
        spk = spike_a[(spike_a>start) & (spike_a<stop)]-start
        spkl = spk.tolist()
        spk_list.append(spkl)
    spk_array = np.asarray(spk_list)
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].spines['left'].set_visible(False)
    ax[0].axes.xaxis.set_visible(False)
    ax[0].eventplot(spk_array, 'horizontal',linelengths=1,linewidths=0.2,color='slategray')
    ax[0].set_ylabel('#Trial',fontsize=14)
    ax[0].set_ylim(-0.5,4.5)
    ax[0].set_yticks(np.arange(0,4.1,1))
    ax[0].set_xlim(0,2*gap)
    ax[0].set_xticks(np.linspace(0,(2*gap/0.2-1)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[0].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    ax[0].set_title(i)

    _hist = np.zeros((len(spk_array), len(np.arange(0, 2*gap, 0.2))-1))
    for row in range(len(spk_array)):
        spk_row = spk_array[row]
        distribution,_ = np.histogram(spk_row, np.arange(0, 2*gap, 0.2))
        _hist[row] = distribution
    hist = np.sum(_hist, 0)
    ax[1].bar((np.arange(0, 2*gap, 0.2)-0.1)[1:], hist, width = 0.2,color='slategray')
    title = '%s_PSTH_5trial.jpg'% i
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['bottom'].set_linewidth('1')
    ax[1].spines['left'].set_linewidth('1')
    ax[1].set_ylabel('Counts',fontsize=14)
    ax[1].set_xlim(0,2*gap)
    ax[1].set_xticks(np.linspace(0,len(hist)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[1].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    plt.savefig(os.path.join(activate_path, title),dpi=400)
    #plt.show()
    plt.close()

## activate-1trial-2s/5s-20s

In [ ]:
gap = 10
marker_a = 20+kbd_a
unit_list = list(wave_a.unit)
unit_name = []
for idx,i in enumerate(unit_list):
    fig,ax = plt.subplots(2,1,figsize=(6,4))
    name = 'timestamp/'+i
    unit_name.append(name)
    spike_a = timestamp_a[name][:].flatten()
    spk_list=[]
    start = marker_a - gap
    stop = marker_a + gap
    spk_array_b = spike_a[(spike_a>start) & (spike_a<start+gap)]-start
    spk_array_a = spike_a[(spike_a>start+gap) & (spike_a<stop)]-start
    spk_array = np.concatenate((spk_array_b,spk_array_a),axis=0)
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].spines['left'].set_visible(False)
    ax[0].axes.xaxis.set_visible(False)
    ax[0].eventplot(spk_array, 'horizontal',linelengths=1,linewidths=0.2,color='slategray')
    #ax[0].set_ylabel('#Trial',fontsize=14)
    #ax[0].set_ylim(-0.5,0.5)
    ax[0].set_yticklabels(['1','',''])
    ax[0].set_xlim(0,2*gap)
    ax[0].set_xticks(np.linspace(0,(2*gap/0.2-1)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[0].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    ax[0].set_title(i)

    _hist = np.zeros((len(spk_array), len(np.arange(0, 2*gap, 0.2))-1))
    for row in range(len(spk_array)):
        spk_row = spk_array[row]
        distribution,_ = np.histogram(spk_row, np.arange(0, 2*gap, 0.2))
        _hist[row] = distribution
    hist = np.sum(_hist, 0)
    ax[1].bar((np.arange(0, 2*gap, 0.2)-0.1)[1:], hist, width = 0.2,color='slategray')
    title = '%s_PSTH.jpg'% i
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['bottom'].set_linewidth('1')
    ax[1].spines['left'].set_linewidth('1')
    ax[1].set_ylabel('Counts',fontsize=14)
    ax[1].set_xlim(0,2*gap)
    ax[1].set_xticks(np.linspace(0,len(hist)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[1].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    plt.savefig(os.path.join(activate_path, title),dpi=400)
    plt.close()